In [1]:
# Data handling libraries
import pandas as pd
import numpy as np

# Web Scraping Libraries
import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
# URL is the base page for the data analyst job on indeed
URL = 'https://ca.indeed.com/jobs?q=data%20analyst&l'

page = requests.get(URL)

In [3]:
# With the help of Beautiful soup function, we are able to generate a human readable html format text from the web page
soup = BeautifulSoup(page.text, 'html.parser')

In [230]:
#Initiating empty lists for the information we will be extracting from web pages.

job_titles = []
job_company = []
job_location = []
job_description = []
jd_urls = []

# Cycling through the pages that contain the job tiles. We are only taking 5 pages here.
for i in range(0, 101, 20):
    # "i" is assigned with each number from the loop
    URL = 'https://ca.indeed.com/jobs?q=data%20analyst&l&start={}'.format(i)
    # requests function will connect to the web page provided the link
    page = requests.get(URL)
    # Beautiful soup function will convert the webpage to a human readable html format
    soup = BeautifulSoup(page.text, 'html.parser')
    # Isolating the job titles in the web page and appending it to the empty list
    for div in soup.find_all(name = 'div', attrs = {'class': 'row'}):
        for a in div.find_all(name = 'a', attrs = {'data-tn-element': 'jobTitle'}):
            job_titles.append(a['title'])
    
    # Isolating the job company in the web page and appending it to the empty list
    for div in soup.find_all(name = 'div', attrs = {'class': 'row'}):
        company = div.find_all(name = 'span', attrs = {'class': 'company'})
        if len(company) > 0:
            for b in company:
                job_company.append(b.text.strip())
        else:
            test = div.find_all(name = 'span', attrs = {'class': 'result-link-source'})
            for span in test:
                job_company.append(span.text.strip)
    
    # Same as the title and company
    for i in soup.find_all():
        if 'data-rc-loc' in i.attrs:
            job_location.append(i['data-rc-loc'])
    
    # Since there is only a short job description in the job tile, we have to open each individual job page to 
    # get the full job descrition
    for div in soup.find_all(name = 'div', attrs = {'class': 'row'}):
        for a in div.find_all(name = 'a', attrs = {'data-tn-element': 'jobTitle'}):
            url_job = 'http://indeed.com' + a['href']
            page_job = requests.get(url_job)
            soup_job = BeautifulSoup(page_job.text, 'html.parser')
            points = soup_job.find_all(name = 'div', attrs = {'class': 'jobsearch-JobComponent-description'})
            points = [j.text for j in points]
            job_description.append('\n'.join(points))
            jd_urls.append(url_job)

In [234]:
# Creating a pandas Data Frame from the finalized lists
jobs_df = pd.DataFrame({'Job Title': job_titles, 'Company': job_company,
                       'Location': job_location, 'Job Description': job_description,
                       'URL': jd_urls})

In [235]:
# Exporting it to a csv file
jobs_df.to_csv('indeed_jobs.csv')